In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import csv

## Authentication

WARNING: Spotify's developer credentials are strictly personal, thus the file containing them will not be made available. Nevertheless, the code will still work as long as a personal "credentials.csv" file is present in the same directory

In [2]:
with open('credentials.csv', mode ='r')as file:
    read_file = csv.reader(file, delimiter=';')

    for credentials in read_file:
        cid = credentials[0]
        secret = credentials[1]

Credentials are managed through Spotipy's built-in functions

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## Playlist extraction

Spotify is organized in playlists, meaning that even data extraction is playlist-based. \
Thus the playlist from which the tracks will be extracted needs to be specified using a Spotify link. \
Playlists are furthermore divided in pages of 100 tracks. If not differently stated only the first page will be read. \
The following function counters this problem:

In [4]:
def get_playlist_tracks(playlist_link):
    playlist_URI = playlist_link.split("/")[-1].split("?")[0]
    results = sp.playlist_tracks(playlist_URI)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [5]:
playlist_link = "https://open.spotify.com/playlist/7ibyZn0ag448uMSkven931?si=0db78178f2b24e36" 

## Data extraction

Hereafter the information on every track present in the playlist is extracted 

In [6]:
df_spoti = pd.DataFrame(columns = ['track_name','artists','album','danceability','energy','key','loudness',
                             'mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                             'duration_ms','time_signature', 'popularity','year'])

for track in get_playlist_tracks(playlist_link):

    track_uri = track["track"]["uri"]
    track_name = track["track"]["name"]
    
    artists_name = ""
    for i, _ in enumerate(track["track"]["artists"]):
        if i == 0:
            artists_name = artists_name + track["track"]["artists"][i]["name"]
        else:
            artists_name = artists_name + ", " + track["track"]["artists"][i]["name"]
    
    album = track["track"]["album"]["name"]
    
    track_popularity = track["track"]["popularity"]
    
    release_year = track["track"]["album"]["release_date"][0:4]
    features = sp.audio_features(track_uri)[0]
    
    entry = pd.DataFrame.from_dict({
        "track_name":  [track_name],
        "artists": [artists_name],
        "album": [album],
        "danceability": [features["danceability"]],
        "energy": [features["energy"]],
        "key": [features["key"]],
        "loudness": [features["loudness"]],
        "mode": [features["mode"]],
        "speechiness": [features["speechiness"]],
        "acousticness": [features["acousticness"]],
        "instrumentalness": [features["instrumentalness"]],
        "liveness": [features["liveness"]],
        "valence": [features["valence"]],
        "tempo": [features["tempo"]],
        "duration_ms": [features["duration_ms"]],
        "time_signature": [features["time_signature"]],
        "popularity": [track_popularity],
        "year": [release_year]
    })
    
    df_spoti = pd.concat([df_spoti, entry], ignore_index=True)

In [7]:
df_spoti.head()

,track_name,artists,album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,year
0,Wouldn't It Be Nice,The Beach Boys,Greatest Hits,0.385,0.657,5,-6.975,1,0.0342,0.724000,0,0.1050,0.672,124.895,153867,4,57,2012
1,God Only Knows,The Beach Boys,50 Big Ones: Greatest Hits,0.433,0.469,6,-9.853,0,0.0320,0.156000,0,0.0479,0.334,115.876,175787,4,49,2012
2,Sinnerman,Nina Simone,Pastel Blues,0.436,0.543,11,-13.351,0,0.0962,0.654000,0.000009,0.0937,0.663,148.282,622000,4,62,1965
3,How to Disappear Completely,Radiohead,Kid A,0.167,0.302,6,-11.644,0,0.0345,0.316000,0.797,0.1100,0.190,102.026,356333,4,66,2000
4,when you sleep,my bloody valentine,loveless,0.399,0.866,6,-9.720,1,0.0558,0.000034,0.738,0.0785,0.125,128.864,251947,4,60,1991


## Merging datasets

Here the dataset is merged with the Rate Your Music top 500 dataset, adding the "ranking" and "critic score" columns

In [12]:
df_rym = pd.read_csv('RYM_top_tracks.csv',delimiter=";")

df = df_spoti.merge(df_rym, how='left', on=['track_name','artists'])

df.to_csv('dataset.csv', sep=';', columns = ['track_name','artists','album','danceability','energy','key','loudness',
                                                         'mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                                         'duration_ms','time_signature', 'popularity','year','ranking','critic score'], index=False)

df.head()

,track_name,artists,album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity,year,ranking,critic score
0,Wouldn't It Be Nice,The Beach Boys,Greatest Hits,0.385,0.657,5,-6.975,1,0.0342,0.724000,0,0.1050,0.672,124.895,153867,4,57,2012,1.0,1
1,God Only Knows,The Beach Boys,50 Big Ones: Greatest Hits,0.433,0.469,6,-9.853,0,0.0320,0.156000,0,0.0479,0.334,115.876,175787,4,49,2012,1.0,1
2,Sinnerman,Nina Simone,Pastel Blues,0.436,0.543,11,-13.351,0,0.0962,0.654000,0.000009,0.0937,0.663,148.282,622000,4,62,1965,2.0,"0,997995992"
3,How to Disappear Completely,Radiohead,Kid A,0.167,0.302,6,-11.644,0,0.0345,0.316000,0.797,0.1100,0.190,102.026,356333,4,66,2000,3.0,"0,995991984"
4,when you sleep,my bloody valentine,loveless,0.399,0.866,6,-9.720,1,0.0558,0.000034,0.738,0.0785,0.125,128.864,251947,4,60,1991,4.0,"0,993987976"
